In [1]:
import json
from pprint import pprint
from data_utils import *

In [2]:
data = super_average_data(process_data())

for condition in data:
    print('condition: ', condition)
    for item in data[condition]:
        print('item: ', item)
        print('sentence: ', data[condition][item]['example_sentence'])
        for measure in data[condition][item]['measurements']:
            print('measure: ', measure, ' = ', data[condition][item]['measurements'][measure])
        print()


/home/lizabialik/projects/701/data_utils.py:117: RuntimeWarning: Mean of empty slice
  return np.round(np.nanmean(values, 0), decimals = 3).tolist()


condition:  ORC
item:  1
sentence:  The bus driver^ that^ the kids^ followed^^ wondered^ about the location of a hotel.
measure:  ff  =  [157.098, 203.836, 222.229, 198.3, 226.664, 245.442, 211.757, nan]
measure:  fp  =  [516.947, 138.28, 310.002, 247.83, 192.313, 285.488, 961.878, 0.0]
measure:  gp  =  [516.947, 152.456, 418.417, 281.272, 200.927, 329.408, 1950.431, 0.0]
measure:  tt  =  [585.92, 231.624, 408.306, 314.964, 230.156, 420.588, 1271.95, 0.0]

item:  2
sentence:  The chef^ that^ the waiter^ distracted^^ poured^ the flour onto the counter.
measure:  ff  =  [181.473, 222.622, 235.431, 238.471, 236.338, 256.838, 211.514, nan]
measure:  fp  =  [298.712, 206.644, 377.381, 361.616, 183.24, 270.199, 979.995, 0.0]
measure:  gp  =  [302.462, 267.929, 598.441, 473.585, 197.466, 380.118, 2207.592, 0.0]
measure:  tt  =  [434.662, 408.652, 707.668, 630.612, 216.977, 386.164, 1228.385, 0.0]

item:  3
sentence:  The children^ that^ the babysitter^ ignored^^ bothered^ her about a trip to 

In [4]:
# # Clone the repository
# !git clone https://github.com/facebookresearch/colorlessgreenRNNs

# # Navigate to the src directory, check its contents
# %cd colorlessgreenRNNs/src
# !ls

# # Get the pretrained model:
# !wget https://dl.fbaipublicfiles.com/colorless-green-rnns/best-models/English/hidden650_batch128_dropout0.2_lr20.0.pt

# !mkdir ../data/lm/English
# !wget -P ../data/lm/English/ https://dl.fbaipublicfiles.com/colorless-green-rnns/training-data/English/train.txt
# !wget -P ../data/lm/English/ https://dl.fbaipublicfiles.com/colorless-green-rnns/training-data/English/test.txt
# !wget -P ../data/lm/English/ https://dl.fbaipublicfiles.com/colorless-green-rnns/training-data/English/valid.txt
# !wget -P ../data/lm/English/ https://dl.fbaipublicfiles.com/colorless-green-rnns/training-data/English/vocab.txt

# # %cd colorlessgreenRNNs/src/language_models


Cloning into 'colorlessgreenRNNs'...
remote: Enumerating objects: 601, done.
remote: Total 601 (delta 0), reused 0 (delta 0), pack-reused 601
Receiving objects: 100% (601/601), 31.27 MiB | 6.63 MiB/s, done.
Resolving deltas: 100% (321/321), done.
Updating files: 100% (86/86), done.
/home/lizabialik/projects/701/colorlessgreenRNNs/src
create_testset.sh  language_models  results.py
data		   README.md	    syntactic_testsets
--2022-05-23 23:30:26--  https://dl.fbaipublicfiles.com/colorless-green-rnns/best-models/English/hidden650_batch128_dropout0.2_lr20.0.pt
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 104.22.74.142, 172.67.9.4, 104.22.75.142, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|104.22.74.142|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 287304504 (274M) [binary/octet-stream]
Saving to: ‘hidden650_batch128_dropout0.2_lr20.0.pt’

hidden650_batch128_ 100%[===================>] 273.99M  11.3MB/s    in 25s     

2022-05

In [1]:
%cd colorlessgreenRNNs/src/language_models

/home/lizabialik/projects/701/colorlessgreenRNNs/src/language_models


In [ ]:
import torch
import torch.nn as nn
import numpy as np

torch.manual_seed(50360)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
np.random.seed(50360)

model_ = None
fn = "../hidden650_batch128_dropout0.2_lr20.0.pt"
with open(fn, "rb") as model_f:
    model_ = torch.load(fn)

if torch.cuda.is_available():
    torch.device('cuda')
else:
    torch.device('cpu')

# Makes sure your model is loaded onto the GPU (should return true)
next(model_.parameters()).is_cuda

# Finally - load the model.
from model import RNNModel

# Construct a new RNNModel using PyTorch 1.x implementations of NN modules
model = RNNModel("LSTM", 50001, 650, 650, 2, 0.2, False)
# Copy over the trained weights from the model loaded in
model.load_state_dict(model_.state_dict())

model = model.cuda()


FileNotFoundError: [Errno 2] No such file or directory: '../hidden650_batch128_dropout0.2_lr20.0.pt'

: 

: 

In [ ]:
## Import dictionary_corpus, part of the colorlessgreenRNNs repoistory that has some use useful functions
import dictionary_corpus

data_path = "../../data/lm/English"
dictionary = dictionary_corpus.Dictionary(data_path)

sentence = "the key to the cabinets"
tokenized_sentence = sentence.split()
input = [dictionary.word2idx[w]  if w in dictionary.word2idx
                                    else dictionary.word2idx["<unk>"]
                  for w in tokenized_sentence]

for w in tokenized_sentence:
    if w not in dictionary.word2idx:
        print(w, ' not in vocab!')

input = torch.tensor(input, dtype=torch.long)
input = input.cuda()

## Extract the hidden and output layers at each input token:
cur_sentence_output, cur_sentence_hidden = model(input.view(-1, 1), # (sequence_length, batch_size).
                       model.init_hidden(1)) # one input at a time, thus batch_size = 1

## Size of the output for the current sentence: Length of the sequence (in words), batch size (1), and length of the vocabulary (50001)
## This is the score of each word in the vocabulary (N = 50001) as the next token in the sequence, for each of the 5 input positions.  
cur_sentence_output.size()

## The output at the final token represents the predictions about the next word in the sequence. 
## We can pull out the next word predictions explicitly, since that's what we care about.
next_word_scores = cur_sentence_output[-1].view(-1)

## What's the most likely next word?
## Get the maximum value, and the associated id, with the max method:
predicted_word, predicted_word_id = next_word_scores.max(0)

predicted_word_id = predicted_word_id.item() # get an int out of a 1x1 tensor. Remove this line to see the difference!
print("Most likely next word ID:",predicted_word_id)
print("Most likely next word:", dictionary.idx2word[predicted_word_id])

## Retrieve the score of a particular word
is_idx = dictionary.word2idx["is"]
are_idx = dictionary.word2idx["are"]

print("is score: ", next_word_scores[is_idx].item())
print("are score: ", next_word_scores[are_idx].item())

## Scores are uninterpretable on their own; we might want to convert them 
## to a probability distribution using softmax
import torch.nn.functional as F

next_word_dist = F.softmax(next_word_scores, dim=0)

print("is prob: ", next_word_dist[is_idx].item())
print("are prob: ", next_word_dist[are_idx].item())


: 

: 